In [ ]:
# ==============================================================================
# 1. SETUP: Install Required Libraries
# ==============================================================================
# Using pyngrok to expose our local server to the internet for webhook callbacks.
%pip install twilio flask python-dotenv pyngrok

In [ ]:
# ==============================================================================
# 2. CONFIGURATION: Load Environment Variables and Initialize Twilio Client
# ==============================================================================
import os
import threading
import time
from dotenv import load_dotenv
from twilio.rest import Client

# Load environment variables from a .env file in the same directory
load_dotenv()

# Fetch credentials from environment variables
ACCOUNT_SID = os.getenv('TWILIO_ACCOUNT_SID')
AUTH_TOKEN = os.getenv('TWILIO_AUTH_TOKEN')
FROM_NUMBER = os.getenv('TWILIO_PHONE_NUMBER')  # Your Twilio number for making calls
TO_NUMBER = os.getenv('INCOMING_PHONE_NUMBER') # The number you will be calling
PORT = 3001  # Port for the Flask server

# Validate that all required variables are set
if not all([ACCOUNT_SID, AUTH_TOKEN, FROM_NUMBER, TO_NUMBER]):
    print("ERROR: Missing one or more required environment variables.")
    print("   Please create a .env file with the following content:")
    print("   TWILIO_ACCOUNT_SID=ACxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
    print("   TWILIO_AUTH_TOKEN=your_auth_token_here")
    print("   TWILIO_PHONE_NUMBER=+15017122661")
    print("   INCOMING_PHONE_NUMBER=+15558675310")
else:
    client = Client(ACCOUNT_SID, AUTH_TOKEN)
    print("Twilio client initialized successfully.")
    print(f"   Account SID: {ACCOUNT_SID[:5]}...")
    print(f"   Outbound Number: {FROM_NUMBER}")
    print(f"   Destination Number: {TO_NUMBER}")


In [ ]:
# ==============================================================================
# 3. WEBHOOK SERVER: Setup Flask and Expose via Ngrok
# ==============================================================================
from flask import Flask, request
from twilio.twiml.voice_response import VoiceResponse
from pyngrok import ngrok

# --- Flask App Definition ---
app = Flask(__name__)

@app.route("/webhook", methods=['POST'])
def handle_webhook():
    """Handles incoming Twilio webhook requests for call status updates."""
    print("\n" + "="*50)
    print("--- WEBHOOK RECEIVED ---")
    print(f"Timestamp: {time.strftime('%Y-%m-%d %H:%M:%S')}")
    # The 'AnsweredBy' parameter is the key to AMD results.
    answered_by = request.form.get('AnsweredBy', 'N/A')
    print(f"Call SID: {request.form.get('CallSid')}")
    print(f"Status: {request.form.get('CallStatus')}")
    print(f"AMD Result: {answered_by}")
    print("="*50)
    print("Full Webhook Data:")
    for key, value in request.form.items():
        print(f"  - {key}: {value}")
    print("="*50 + "\n")
    return str(VoiceResponse()), 200

@app.route("/handle_amd", methods=['POST'])
def handle_amd_twiml():
    """Generates TwiML based on the AMD result. Used by Use Case #5."""
    response = VoiceResponse()
    answered_by = request.form.get('AnsweredBy', 'unknown')

    print(f"\n" + "="*50)
    print(f"TwiML Handler /handle_amd triggered! AMD Result: {answered_by}")
    print(f"Call SID: {request.form.get('CallSid')}")
    print("="*50 + "\n")

    if answered_by in ['machine_start', 'machine_end_beep', 'machine_end_silence', 'machine_end_other']:
        response.say("This is a message for your answering machine. Have a great day!", voice='alice')
        response.hangup()
    elif answered_by == 'human':
        response.say("Hello, a human has answered. This is a test from your Twilio demo.", voice='alice')
    elif answered_by == 'fax':
        response.say("A fax machine was detected.", voice='alice')
        response.hangup()
    else: # unknown or other cases
        response.say("Could not determine if human or machine. Ending call.", voice='alice')

    return str(response)

# --- Server and Ngrok Tunnel Control ---
def run_app():
    # Note: Using 'werkzeug' reloader is problematic with ngrok in notebooks.
    app.run(host='0.0.0.0', port=PORT, debug=False)

def start_server_and_ngrok():
    """Starts the Flask server in a thread and creates a public ngrok tunnel."""
    # Start Flask server in a background thread
    flask_thread = threading.Thread(target=run_app, daemon=True)
    flask_thread.start()
    time.sleep(1) # Give the server a moment to start

    # Start ngrok tunnel
    try:
        # You may need to add your ngrok authtoken first: `ngrok config add-authtoken <YOUR_TOKEN>`
        public_url = ngrok.connect(PORT)
        print("\nWebhook server is running and exposed via ngrok.")
        print(f"   Public URL: {public_url}")
        return public_url
    except Exception as e:
        print(f"ERROR: Could not start ngrok. Please ensure ngrok is installed and configured.")
        print(f"   Error details: {e}")
        return None

def stop_server_and_ngrok(public_url):
    """Shuts down the specified ngrok tunnel."""
    if not public_url:
        print("\nNo active tunnel to shut down.")
        return
        
    print("\nShutting down ngrok tunnel...")
    ngrok.disconnect(public_url)
    print("   Tunnel closed.")

# --- Start the server and get the public URL for our demo ---
PUBLIC_URL = start_server_and_ngrok()

In [22]:
# ==============================================================================
# 4. DEMO LOGIC: Define AMD Use Cases and Call Function
# ==============================================================================
if PUBLIC_URL:
    # Dynamically update webhook URLs with our new public ngrok address
    AMD_USE_CASES = {
        "1": {
            "name": "Standard AMD (Detect MessageEnd)",
            "description": "Uses 'machine_detection=DetectMessageEnd' to wait for the beep.",
            "params": {
                "machine_detection": "DetectMessageEnd",
            }
        },
        "2": {
            "name": "Enable AMD (Basic)",
            "description": "Uses 'machine_detection=Enable' for faster human/machine determination.",
            "params": {
                "machine_detection": "Enable",
            }
        },
        "3": {
            "name": "High Human Sensitivity",
            "description": "Adjusts timeouts to be more sensitive to human speech.",
            "params": {
                "machine_detection": "Enable",
                "machine_detection_speech_threshold": 1800, # Lower threshold requires less speech time
                "machine_detection_speech_end_threshold": 800,
                "machine_detection_silence_timeout": 4000
            }
        },
        "4": {
            "name": "High Machine Sensitivity",
            "description": "Adjusts timeouts to better identify machine greetings.",
            "params": {
                "machine_detection": "Enable",
                "machine_detection_speech_threshold": 3000, # Higher threshold expects more speech (voicemail prompt)
                "machine_detection_speech_end_threshold": 1500,
                "machine_detection_silence_timeout": 6000
            }
        },
        "5": {
            "name": "AMD with TwiML Handler",
            "description": "Uses 'machine_detection=Enable' and directs Twilio to a TwiML URL for custom logic.",
            "params": {
                "machine_detection": "Enable",
                "url": f"{PUBLIC_URL}/handle_amd", # Use our ngrok URL
            }
        }
    }

def make_amd_call(use_case_id):
    """Makes an outbound call using a selected AMD configuration."""
    if not PUBLIC_URL:
        print("ERROR: Cannot make call: Webhook server is not running.")
        return

    if use_case_id not in AMD_USE_CASES:
        print(f"ERROR: Invalid use case ID: {use_case_id}")
        return

    use_case = AMD_USE_CASES[use_case_id]
    print(f"\n" + "-"*60)
    print(f"Initiating Call for Use Case #{use_case_id}: {use_case['name']}")
    print(f"   Description: {use_case['description']}")
    print(f"   Parameters: {use_case['params']}")
    print(f"   Watching for callbacks at: {PUBLIC_URL}/webhook")
    print("-"*60)


    try:
        call_params = {
            'to': TO_NUMBER,
            'from_': FROM_NUMBER,
            # This is the crucial callback that will receive the AMD result
            'async_amd_status_callback': f"{PUBLIC_URL}/webhook",
            'async_amd_status_callback_method': 'POST',
            **use_case['params'] # Unpack the specific params for this use case
        }

        # If a TwiML URL isn't specified, provide a default one.
        if 'url' not in call_params:
             call_params['twiml'] = '<Response><Say>Hello, this is a test call.</Say><Pause length="10"/></Response>'

        call = client.calls.create(**call_params)

        print("Call initiated successfully!")
        print(f"   Call SID: {call.sid}")
        print("\nWaiting for call to connect... Watch this console for real-time webhook updates!")

    except Exception as e:
        print(f"ERROR making call: {e}")

In [ ]:
# ==============================================================================
# 5. INTERACTIVE DEMO: Run the Test
# ==============================================================================
def interactive_amd_test():
    """Presents an interactive menu to test different AMD scenarios."""
    if not PUBLIC_URL:
        print("Interactive demo cannot start because the webhook server failed to initialize.")
        return

    while True:
        print("\n" + "="*50)
        print("          Twilio AMD Interactive Demo")
        print("="*50)
        for key, case in AMD_USE_CASES.items():
            print(f"  {key}. {case['name']}")
        print("  0. Exit")
        print("-"*50)

        choice = input("Enter your choice (0-5): ").strip()

        if choice == "0":
            break
        elif choice in AMD_USE_CASES:
            make_amd_call(choice)
            # Give the user time to see the results before re-prompting
            input("\nPress Enter to return to the menu...")
        else:
            print("ERROR: Invalid choice! Please select a number from the menu.")

    print("Demo finished.")

# --- Run the interactive test ---
interactive_amd_test()

In [ ]:
# ==============================================================================
# 6. CLEANUP: Shut Down the Server and Tunnel
# ==============================================================================
# It's important to run this when you're done to close the public connection.
stop_server_and_ngrok(PUBLIC_URL)